In [46]:
import sys
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QLabel, QLineEdit, QRadioButton, QGroupBox, QHBoxLayout, QDateTimeEdit, QMainWindow, QCheckBox
from PyQt5.QtCore import QDateTime
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

In [58]:
class Window_2D(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("2D Model Input")
        layout = QVBoxLayout()

        #Define Ellipse Sectoin
        ellipsebox = QGroupBox("Ellipse Parameters")
        ellipse_layout = QVBoxLayout()

        self.centre_layout = QHBoxLayout()
        self.centre_layout.addWidget(QLabel("Centre (x,y):"))
        self.centre_text = QLineEdit("(0,0)")
        self.centre_layout.addWidget(self.centre_text)

        self.width_layout = QHBoxLayout()
        self.width_layout.addWidget(QLabel("Width:"))
        self.width_text = QLineEdit("6378137")
        self.width_layout.addWidget(self.width_text)

        self.height_layout = QHBoxLayout()
        self.height_layout.addWidget(QLabel("Height:"))
        self.height_text = QLineEdit("6378137")
        self.height_layout.addWidget(self.height_text)

        self.angle_layout = QHBoxLayout()
        self.angle_layout.addWidget(QLabel("Angle:"))
        self.angle_text = QLineEdit("0")
        self.angle_layout.addWidget(self.angle_text)

        ellipse_layout.addLayout(self.centre_layout)
        ellipse_layout.addLayout(self.width_layout)
        ellipse_layout.addLayout(self.height_layout)
        ellipse_layout.addLayout(self.angle_layout)
        ellipsebox.setLayout(ellipse_layout)

        layout.addWidget(ellipsebox)


        #Satellite Section
        satellitebox = QGroupBox("Satellite Parameters")
        satellite_layout = QVBoxLayout()

        self.mass_layout = QHBoxLayout()
        self.mass_layout.addWidget(QLabel("Mass (kg):"))
        self.mass_text = QLineEdit("3000")
        self.mass_layout.addWidget(self.mass_text)

        self.drag_layout = QHBoxLayout()
        self.drag_layout.addWidget(QLabel("Drag Coefficient:"))
        self.drag_text = QLineEdit("2.2")
        self.drag_layout.addWidget(self.drag_text)

        self.initpos_layout = QHBoxLayout()
        self.initpos_layout.addWidget(QLabel("Initial Position:"))
        self.initpos_text = QLineEdit()
        self.initpos_layout.addWidget(self.initpos_text)
        self.default_initpos = QPushButton("Sample")
        self.default_initpos.clicked.connect(self.set_initpos)
        self.initpos_layout.addWidget(self.default_initpos)

        self.initspeed_layout = QHBoxLayout()
        self.initspeed_layout.addWidget(QLabel("Initial Speed (Clockwise):"))
        self.initspeed_text = QLineEdit()
        self.initspeed_layout.addWidget(self.initspeed_text)

        self.initveloc_norm_layout = QHBoxLayout()
        self.initveloc_norm_layout.addWidget(QLabel("Initial Direction:"))
        self.initveloc_norm_text = QLineEdit()
        self.initveloc_norm_layout.addWidget(self.initveloc_norm_text)
        self.tangent_velocity_flag = QCheckBox("Auto Tangent")
        self.tangent_velocity_flag.stateChanged.connect(self.tangent_velocity)
        self.initveloc_norm_layout.addWidget(self.tangent_velocity_flag)

        self.inittime_layout = QHBoxLayout()
        self.inittime_layout.addWidget(QLabel("Initial Time:"))
        self.inittime_text = QDateTimeEdit()
        self.inittime_layout.addWidget(self.inittime_text)

        satellite_layout.addLayout(self.mass_layout)
        satellite_layout.addLayout(self.drag_layout)
        satellite_layout.addLayout(self.initpos_layout)
        satellite_layout.addLayout(self.initspeed_layout)
        satellite_layout.addLayout(self.initveloc_norm_layout)
        satellite_layout.addLayout(self.inittime_layout)
        satellitebox.setLayout(satellite_layout)

        layout.addWidget(satellitebox)


        #Radar Section
        radarbox = QGroupBox("Radar Parameters")
        radar_layout = QVBoxLayout()

        self.radarcomplexity = QHBoxLayout()
        self.radiosimple = QRadioButton("Simple Radar")
        self.radiocomplex = QRadioButton("Complex Radar")
        self.radiosimple.setChecked(True)
        self.radarcomplexity.addWidget(self.radiosimple)
        self.radarcomplexity.addWidget(self.radiocomplex)

        self.radarparam_layout = QHBoxLayout()
        self.radarparam_layout.addWidget(QLabel("Radar Numbers (simple)/Locations (complex):"))
        self.radarparam_text = QLineEdit("8")
        self.radarparam_layout.addWidget(self.radarparam_text)

        self.noiselevel_layout = QHBoxLayout()
        self.noiselevel_layout.addWidget(QLabel("Radar Noise (%):"))
        self.noiselevel_text = QLineEdit("0.05")
        self.noiselevel_layout.addWidget(self.noiselevel_text)

        self.readint_layout = QHBoxLayout()
        self.readint_layout.addWidget(QLabel("Readings Interval (s):"))
        self.readint_text = QLineEdit("10")
        self.readint_layout.addWidget(self.readint_text)

        radar_layout.addLayout(self.radarcomplexity)
        radar_layout.addLayout(self.radarparam_layout)
        radar_layout.addLayout(self.noiselevel_layout)
        radar_layout.addLayout(self.readint_layout)
        radarbox.setLayout(radar_layout)

        layout.addWidget(radarbox)


        #Simulator Parameters
        simulatorbox = QGroupBox("Simulator Parameters")
        simulator_layout = QVBoxLayout()

        self.steptime_layout = QHBoxLayout()
        self.steptime_layout.addWidget(QLabel("Time step dt (s):"))
        self.steptime_text = QLineEdit("0.1")
        self.steptime_layout.addWidget(self.steptime_text)

        self.maxiter_layout = QHBoxLayout()
        self.maxiter_layout.addWidget(QLabel("Maximum Iterations:"))
        self.maxiter_text = QLineEdit("1000000")
        self.maxiter_layout.addWidget(self.maxiter_text)

        self.simple_solver_layout = QHBoxLayout()
        self.simple_solver = QCheckBox("Simple Forward Euler Solver (Not Recommended)")
        self.simple_solver_layout.addWidget(self.simple_solver)

        simulator_layout.addLayout(self.steptime_layout)
        simulator_layout.addLayout(self.maxiter_layout)
        simulator_layout.addLayout(self.simple_solver_layout)
        simulatorbox.setLayout(simulator_layout)
        
        layout.addWidget(simulatorbox)

        button_layout = QHBoxLayout()
        self.button_back = QPushButton("Back")
        self.button_back.clicked.connect(self.go_back)
        self.button_confirm = QPushButton("Confirm")

        button_layout.addWidget(self.button_back)
        button_layout.addWidget(self.button_confirm)
        layout.addLayout(button_layout)
        
        self.setLayout(layout)
    
    def go_back(self):
        self.w = MainWindow()
        self.w.show()
        self.close()
    
    def set_initpos(self):
        if self.width_text.text() != "" and self.height_text != "" and self.centre_text != "":
            print(f"{((eval(self.width_text.text()) - eval(self.centre_text.text())[0])) + 0.1 * eval(self.width_text.text())}")
            self.initpos_text.setText(f"[{((eval(self.width_text.text()) - eval(self.centre_text.text())[0])) + 0.1 * eval(self.width_text.text())}, {((eval(self.height_text.text()) - eval(self.centre_text.text())[1])) + 0.1 * eval(self.height_text.text())}]")
        else:
            self.initpos_text.setText("[100,100]")

    def tangent_velocity(self):
        if self.tangent_velocity_flag.isChecked():
            self.initveloc_norm_text.setText("")
            self.initveloc_norm_text.setEnabled(False)
        else:
            self.initveloc_norm_text.setEnabled(True)


class Window_3D(QWidget):
    def __init__(self):
        super().__init__()
        layout = QVBoxLayout()
        self.label = QLabel("3D Selector!")

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.w = None

        self.setWindowTitle("Skylab Orbit Predictor GUI")
        
        layout = QVBoxLayout()

        self.button_2D = QPushButton("2D Version")
        self.button_2D.clicked.connect(self.show_2D_window)
        self.button_3D = QPushButton("3D Version")
        self.button_3D.clicked.connect(self.show_3D_window)
        
        layout.addWidget(self.button_2D)
        layout.addWidget(self.button_3D)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
    
    def show_2D_window(self, checked):
        self.w = Window_2D()
        self.w.show()
        self.close()
    
    def show_3D_window(self, checked):
        self.w = Window_2D()
        self.w.show()
        self.close()

if __name__ == '__main__':
    app = 0
    app = QApplication(sys.argv)
    w = MainWindow()
    w.show()
    sys.exit(app.exec_())

In [3]:
class App(QWidget):
    def __init__(self):
        super().__init__()
        self.title = 'Satellite Trajectory GUI'
        self.initUI()
        
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(100, 100, 340, 260)
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        # Model selection
        model_group_box = QGroupBox("Model Selection")
        model_layout = QHBoxLayout()
        
        self.push2D = QPushButton("2D Model")
        self.push2D.clicked.connect(self.transfer_2D())
        self.push3D = QPushButton("3D Model")
        self.push3D.clicked.connect(self.transfer_2D())

        model_layout.addWidget(self.push2D)
        model_layout.addWidget(self.push3D)
        model_group_box.setLayout(model_layout)
        layout.addWidget(model_group_box)
    
    def transfer_2D(self):
        self.close()
        self.Open = App().click_2D()
        self.Open.show()
    
    def transfer_back(self):
        self.close()
        self.Open = App().initUI()
        self.Open.show()
    
    def click_2D(self):
        self.setWindowTitle("2D Model Selection")
        self.setGeometry(100, 100, 340, 260)

        layout = QVBoxLayout()
        self.setLayout(layout)

        self.pushBack = QPushButton("Back")
        self.pushBack.clicked.connect(self.transfer_back(flag='init'))
        layout.addWidget(self.pushBack)
    #     pass
    #     model_layout.addWidget(self.radio2D)
    #     model_layout.addWidget(self.radio3D)
    #     model_group_box.setLayout(model_layout)
        
    #     # Parameter inputs
    #     param_layout = QVBoxLayout()
    #     self.velocity_input = QLineEdit(self)
    #     self.mass_input = QLineEdit(self)
    #     self.drag_coeff_input = QLineEdit(self)
        
    #     self.start_time_input = QDateTimeEdit(QDateTime.currentDateTime(), self)
    #     self.start_time_input.setDisplayFormat("yyyy:MM:dd HH:mm:ss")
        
    #     param_layout.addWidget(QLabel("Velocity:"))
    #     param_layout.addWidget(self.velocity_input)
    #     param_layout.addWidget(QLabel("Mass:"))
    #     param_layout.addWidget(self.mass_input)
    #     param_layout.addWidget(QLabel("Drag Coefficient:"))
    #     param_layout.addWidget(self.drag_coeff_input)
    #     param_layout.addWidget(QLabel("Start Time:"))
    #     param_layout.addWidget(self.start_time_input)
        
    #     # Submit button
    #     submit_btn = QPushButton('Run Simulation', self)
    #     submit_btn.clicked.connect(self.on_click)
        
    #     layout.addWidget(model_group_box)
    #     layout.addLayout(param_layout)
    #     layout.addWidget(submit_btn)
    
    # def on_click(self):
    #     model = '2D' if self.radio2D.isChecked() else '3D'
    #     velocity = self.velocity_input.text()
    #     mass = self.mass_input.text()
    #     drag_coeff = self.drag_coeff_input.text()
    #     start_time = self.start_time_input.dateTime().toString("yyyy:MM:dd HH:mm:ss")
        
    #     if model == "2D":
    #         Simulator_2D()
    #         Predictor_2D()
    #     else:
    #         Simulator()
    #         Predictor()
        
    #     print(f"Running with Model: {model}, Velocity: {velocity}, Mass: {mass}, Drag Coeff: {drag_coeff}, Start Time: {start_time}")
        
    #     self.vis_window = VisualizationWindow(model)
    #     self.vis_window.show()
        
class VisualizationWindow(QMainWindow):
    def __init__(self, model):
        super().__init__()
        self.title = 'Simulation and Prediction Visualization'
        self.model = model
        self.initUI()

    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(100, 100, 800, 600)
        
        widget = QWidget(self)
        self.setCentralWidget(widget)
        layout = QVBoxLayout()
        widget.setLayout(layout)
        
        self.figure = plt.figure()
        self.canvas = FigureCanvas(self.figure)
        layout.addWidget(self.canvas)

        if self.model == "3D":
            self.ax = self.figure.add_subplot(111, projection='3d')
            self.draw_3d_plot()
        else:
            self.ax = self.figure.add_subplot(111)
            self.draw_2d_plot()

    def draw_plot_2d(self):
        sim_data = pd.read_csv('simulator_2D.csv')
        pred_data = pd.read_csv('predictor_2D.csv')

        earth = plt.Circle((0, 0), 1, color='blue', label='Earth')
        self.ax.add_patch(earth)

        self.ax.plot(sim_data['x'], sim_data['y'], 'ro-', label='Satellite Path')
        
        self.ax.plot(pred_data['x'], pred_data['y'], 'go--', label='Predicted Path')

        self.ax.legend()
        self.ax.grid(True)
        self.ax.set_xlabel('X Coordinate')
        self.ax.set_ylabel('Y Coordinate')
        self.ax.set_title('Satellite Trajectory and Prediction')
        self.canvas.draw()

    def draw_3d_plot(self):
        u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
        x = np.cos(u)*np.sin(v)
        y = np.sin(u)*np.sin(v)
        z = np.cos(v)
        self.ax.plot_surface(x, y, z, color='b', alpha=0.6)

        sim_data = pd.read_csv('simulator.csv')
        x, y, z = sim_data['x'], sim_data['y'], sim_data['z']

        self.sc = self.ax.scatter(x[0], y[0], z[0], color='r')

        def update(frame):
            self.sc._offsets3d = (x[frame:frame+1], y[frame:frame+1], z[frame:frame+1])
            return self.sc,

        self.ani = FuncAnimation(self.figure, update, frames=len(x), interval=50, blit=False)
        self.canvas.draw()

if __name__ == '__main__':
    app = 0
    app = QApplication(sys.argv)
    ex = App()
    ex.show()
    sys.exit(app.exec_())

RecursionError: maximum recursion depth exceeded while calling a Python object